In [13]:
import pandas as pd
import parselmouth
import numpy as np

zoom3 = pd.read_csv(r"C:\Users\IEC010240\Desktop\Thai_Tone\ZOOM0003.txt", encoding="UTF-8", sep="\t", header=0)
#df['tmin'] = df['tmin'].str.replace(" ", "")
#df['tmax'] = df['tmax'].str.replace(" ", "")
zoom3['tier'] = zoom3['tier'].str.replace(" ", "")
zoom3['text'] = zoom3['text'].str.replace(" ", "")
#print("zoom3")
#print(zoom3)
#print("----")

zoom4 = pd.read_csv(r"C:\Users\IEC010240\Desktop\Thai_Tone\ZOOM0004.txt", encoding="UTF-8", sep=" ", header=0)
#df['tmin'] = df['tmin'].str.replace(" ", "")
#df['tmax'] = df['tmax'].str.replace(" ", "")
zoom4['tier'] = zoom4['tier'].str.replace(" ", "")
zoom4['text'] = zoom4['text'].str.replace(" ", "")
#print("zoom4")
#print(zoom4)
#print("----")

zoom5 = pd.read_csv(r"C:\Users\IEC010240\Desktop\Thai_Tone\ZOOM0005.txt", encoding="UTF-8", sep="\t", header=0)
#df['tmin'] = df['tmin'].str.replace(" ", "")
#df['tmax'] = df['tmax'].str.replace(" ", "")
zoom5['tier'] = zoom5['tier'].str.replace(" ", "")
zoom5['text'] = zoom5['text'].str.replace(" ", "")
#print("zoom5")
#print(zoom5)
#print("----")

zoom3snd = parselmouth.Sound(r"C:\Users\IEC010240\Desktop\Thai_Tone\ZOOM0003.WAV")
zoom4snd = parselmouth.Sound(r"C:\Users\IEC010240\Desktop\Thai_Tone\ZOOM0004.WAV")
zoom5snd = parselmouth.Sound(r"C:\Users\IEC010240\Desktop\Thai_Tone\ZOOM0005.WAV")

In [14]:
def aggdf(df):
    column=['tmin','tmax','word','IU/tone','EU/tone', 'pMax', 'pMin', 'pAvg']
    newdf = pd.DataFrame(columns = column)
    #print(newdf)
    #print(int(df.shape[0]-1 / 3))

    for i in range(df.shape[0]):
        tmin = df.iloc[i].tmin
        tmax = df.iloc[i].tmax
        r = newdf.loc[(newdf['tmax'] == tmax) & (newdf['tmin'] == tmin)]
        #print(r)
        if r.empty:
            s = pd.Series({'tmin':tmin, 'tmax':tmax})
            # 这里 Series 必须是 dict-like 类型
            newdf = newdf.append(s, ignore_index=True)    

    for idxbase in range(df.shape[0]):
        tmin = df.iloc[idxbase].tmin
        tmax = df.iloc[idxbase].tmax

        if df.iloc[idxbase].tier =='word':
            #print(idxbase)
            word = df.iloc[idxbase].text
            word = word.replace("?","")
            word = word.replace("*","")
            newdf.loc[(newdf.tmax == tmax) & (newdf.tmin == tmin), 'word'] = word
        elif df.iloc[idxbase].tier =='IU/tone':
            #print(idxbase)
            tiu = df.iloc[idxbase].text.replace("?","")
            if len(df.iloc[idxbase].text)<= 2 :
                iu = tiu
            else:
                s = tiu
                iu = " ".join(s[k:k+2] for k in range(0, len(s), 2))
            newdf.loc[(newdf.tmax == tmax) & (newdf.tmin == tmin), 'IU/tone'] = iu
        elif df.iloc[idxbase].tier =='EU/tone':
            #print("EU/tone", df.iloc[idxbase].tier)
            teu = df.iloc[idxbase].text.replace("?","")
            if len(df.iloc[idxbase].text) <= 2 :
                eu = teu
            else:
                s = teu
                eu = " ".join(s[k:k+2] for k in range(0, len(s), 2))
            newdf.loc[(newdf.tmax == tmax) & (newdf.tmin == tmin), 'EU/tone'] = eu

    #pd.set_option('display.width', 1000)
    #newdf.style.set_properties(**{'text-align': 'right'})
    #print(newdf)
    return newdf

In [18]:
def extract_pitch(snd, tmin, tmax):
    snd_part = snd.extract_part(from_time=tmin, to_time=tmax, preserve_times=True)
    pch = snd_part.to_pitch()
    #print(pch)
    pay = pch.to_matrix()
    a = pay.as_array()
    aa = a[a != 0]
    if len(aa) != 0:
        pmax = np.max(aa)
        pmin = np.min(aa)
        pavg = np.average(aa)
        #print(pmax, pmin, aa, pavg)
        return pmax, pmin, pavg
    else:
        return np.nan, np.nan, np.nan


In [19]:
def dfpitch(df, snd):
    #print(df)
    for index, row in df.iterrows():
        if row["IU/tone"] == row["EU/tone"] and row["word"] != "":
            #print(i, "Equal")
            pmax, pmin, pavg = extract_pitch(snd, row["tmin"], row["tmax"])
            #print(i, df.iloc[i]["tmin"], df.iloc[i]["tmax"], pmin, pmax, pavg)
            df.set_value(index,'pMax', pmax)
            df.set_value(index,'pMin', pmin)
            df.set_value(index,'pAvg', pavg)
    return df

In [6]:
newzoom3=aggdf(zoom3)

In [7]:
z3 = dfpitch(newzoom3, zoom3snd)
z3

C:\Users\IEC010240\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\IEC010240\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
C:\Users\IEC010240\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


,tmin,tmax,word,IU/tone,EU/tone,pMax,pMin,pAvg
0,4.509497,5.742173,老師,21 55,51 55,NaN,NaN,NaN
1,17.523613,19.394128,老師,21 55,51 55,NaN,NaN,NaN
2,25.184943,26.489286,海洋,21 35,21 35,266.479102,76.999431,128.063852
3,56.586235,58.535584,,NaN,NaN,NaN,NaN,NaN
4,62.123764,63.944112,,NaN,NaN,NaN,NaN,NaN
5,67.746904,68.456410,,NaN,NaN,NaN,NaN,NaN
6,74.316441,75.377116,老師,21 55,55 51,NaN,NaN,NaN
7,78.693606,79.180943,,NaN,NaN,NaN,NaN,NaN
8,89.787953,90.031417,我,21,55,NaN,NaN,NaN
9,90.031417,90.429474,簽名,55 35,55 51,NaN,NaN,NaN


In [8]:
newzoom4 = aggdf(zoom4)

In [9]:
z4 = dfpitch(newzoom4, zoom4snd)
z4

C:\Users\IEC010240\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\IEC010240\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
C:\Users\IEC010240\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


,tmin,tmax,word,IU/tone,EU/tone,pMax,pMin,pAvg
0,52.601,53.022,看,51,51,532.028699,220.006567,399.892397
1,53.022,53.751,老師,21 55,21 55,448.209673,95.473061,252.316610
2,54.845,56.052,閉起來,51 21 35,51 21 35,531.267484,136.166971,365.883868
3,61.272,62.282,一人一分,51 35 51 55,51 35 21 55,NaN,NaN,NaN
4,100.896,102.143,影子,21,21,318.896064,95.514681,178.071639
5,103.352,104.292,影子,21,21,280.955108,84.275289,208.790906
6,115.338,116.604,瓶子,35,35,485.662693,257.888180,326.378279
7,117.861,118.868,瓶子,35,35,304.788368,92.909001,208.515031
8,122.811,123.617,喝,55,55,326.183854,94.982682,201.764756
9,123.617,124.519,水,21,21,364.438651,104.203211,288.764855


In [20]:
newzoom5 = aggdf(zoom5)

0
5
6
9
12
16
18
23
26
29
32
34
38
40
42
47
50
53
56
57
61
65
68
70
73
76
80
83
86
89
92
93
98
101
104
107
108
111
115
117
120
124
127
131
132
137
138
142
145
149
150
154
156
159
163
165
169
171
176
179
181
185
186
191
193
196
200
203
204
208
212
213
217
220
224
227
230
231
234
237
242
245
248
249
254
255
260
262
266
268
270
274
278
281
283
285
290
292
295
298
300
303
306
308
310
313
317
321
324
326
328
333
335
338
340
344
346
349
354
357
359
361
365
368
370
375
378
381
384
387
388
393
394
399
401
405
407
410
412
417
419
422
424
428
430
435
438
439
443
447
449
451
455
458
461
464
466
471
472
477
480
481
484
488
492
493
496
500
504
507
510
512
515
519
520
523
526
531
534
537
539
542
544
549
552
554
558
559
562
567
570
573
574
577
580
583
586
590
593
596
598
603
605
608
612
614
616
621
622
626
629
632
636
637
641
644
648
649
652
656
660
663
666
668
670
673
677
679
682
685
690
692
696
699
700
703
708
710
713
717
718
721
724
729
731
733
736
739
742
747
750
753
755
757
760
764
768
769
773
7

In [21]:
z5 = dfpitch(newzoom5, zoom5snd)
z5

C:\Users\IEC010240\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\IEC010240\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
C:\Users\IEC010240\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


,tmin,tmax,word,IU/tone,EU/tone,pMax,pMin,pAvg
0,32.105275,32.974001,老,21,35,NaN,NaN,NaN
1,35.714925,36.483709,老師,21 55,21 55,527.823438,109.380469,237.587659
2,38.023245,38.576770,老師,21 55,21 55,181.230490,129.446429,157.302417
3,38.576770,38.876595,我,21,21,261.266530,174.167330,214.778122
4,38.876595,39.253300,要,51,51,563.222361,175.632184,331.175108
5,39.253300,40.421852,喝水,55 21,55 21,571.335502,127.721312,319.290137
6,137.425903,138.263878,老師,21 55,21 55,351.630309,88.467592,175.705379
7,138.263878,138.579079,我,21,21,124.528575,92.176244,115.409809
8,138.579079,139.024974,要,51,51,348.662424,281.905078,305.724657
9,139.024974,139.816822,喝水,55 21,55 21,373.268969,81.231884,291.753338


In [ ]:
#!/usr/bin/env python
import numpy as np
import matplotlib.pyplot as plt

# example data
x = np.arange(0.1, 4, 0.5)
y = np.exp(-x)

# example variable error bar values
yerr = 0.1 + 0.2*np.sqrt(x)
xerr = 0.1 + yerr

# First illustrate basic pyplot interface, using defaults where possible.
plt.figure()
plt.errorbar(x, y, xerr=0.2, yerr=0.4)
plt.title("Simplest errorbars, 0.2 in x, 0.4 in y")

# Now switch to a more OO interface to exercise more features.
fig, axs = plt.subplots(nrows=2, ncols=2, sharex=True)
ax = axs[0,0]
ax.errorbar(x, y, yerr=yerr, fmt='o')
ax.set_title('Vert. symmetric')

# With 4 subplots, reduce the number of axis ticks to avoid crowding.
ax.locator_params(nbins=4)

ax = axs[0,1]
ax.errorbar(x, y, xerr=xerr, fmt='o')
ax.set_title('Hor. symmetric')

ax = axs[1,0]
ax.errorbar(x, y, yerr=[yerr, 2*yerr], xerr=[xerr, 2*xerr], fmt='--o')
ax.set_title('H, V asymmetric')

ax = axs[1,1]
ax.set_yscale('log')
# Here we have to be careful to keep all y values positive:
ylower = np.maximum(1e-2, y - yerr)
yerr_lower = y - ylower

ax.errorbar(x, y, yerr=[yerr_lower, 2*yerr], xerr=xerr,
            fmt='o', ecolor='g', capthick=2)
ax.set_title('Mixed sym., log y')

fig.suptitle('Variable errorbars')

plt.show()